<a id="readme-top"></a>

# LP Presolve Testground

## Table of Contents
- [Table of Contents](#table-of-contents)
- [Introduction](#introduction)
    - <a href="#andersons-paper">Presolving in Linear Programming Routine</a>
    - <a href="#lp_solve-presolve-routine">lp_solve Presolve Routine</a>
    - <a href="#highs-presolve-routine">HiGHS Presolve Routine</a>
    - <a href="#glpk-presolve-routine">GLPK Presolve Routine</a>
- [Packages](#packages)
- [Modified LPProblem Structs](#modified-lpproblem-structs)
- [Test Problems](#test-problems)
- [Presolve Features](#presolve-features)
    - [Removing Empty Rows](#removing-empty-rows)
    - [Removing Empty Columns](#removing-empty-columns)
    - [Remove Linear Dependent Rows](#remove-linear-dependent-rows)
    - [Tightening Constraints](#tightening-constraints)
- [General Presolve Routine](#general-presolve-routine)
- [Presolve Methods Checklist](#presolve-methods-checklist)
- [References](#references)


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Introduction

Presolving is a vital task of high performance LP solvers as it minimizes the size of the working problem.

<details>
    <summary><h3 id="andersons-paper"> Anderson's Paper </h3></summary>

**Procedure Presolve**
- Remove all fixed variables.
- **repeat**
    - **Check rows:**
        - Remove all row singletons.
        - Remove all forcing constraints.
        - Remove all dominated constraints.
    - **Check columns:**
        - Remove all free, implied free column singletons and all column singletons in combination with a doubleton equation.
        - Remove all dominated columns.
    - **Check for duplicates:**
        - Remove duplicate rows.
        - Remove duplicate columns.
- **until** no reductions in last pass.
- Remove all empty rows and columns.

**end Presolve.**
<sup id="cite1">[Anderson and Anderson, 1995](#ref1)</sup>

![Anderson's Routine](../../diagrams/Andersons-routine.png)


</details>

<details>
    <summary><h3 id="lp_solve-presolve-routine"> lp_solve Presolve Routine </h3></summary>

**Procedure lp_solve Presolve**
- Remove all fixed variables.
- **repeat**
    - **Check rows:**
        - Remove all row singletons.
        - Remove all forcing constraints.
        - Remove all dominated constraints.
    - **Check columns:**
        - Remove all free, implied free column singletons and all column singletons in combination with a doubleton equation.
        - Remove all dominated columns.
    - **Check for duplicates:**
        - Remove duplicate rows.
        - Remove duplicate columns.
    - **Bound tightening:**
        - Tighten bounds using dual information and constraint propagation.
    - **Implied equalities:**
        - Detect and process implied equalities.
    - **Sparsity enhancement:**
        - Enhance matrix sparsity through aggregation and other techniques.
    - **Scaling:**
        - Apply row and column scaling for numerical stability.
- **until** no reductions in last pass.
- Remove all empty rows and columns.

**end lp_solve Presolve.**
<sup id="cite2">[Eikland & Notebaert, 2020](#ref2)</sup>

</details>

<details>
    <summary><h3 id="glpk-presolve-routine"> GLPK Presolve Routine </h3></summary>

**Procedure GLPK Presolve**
- **Initialization:**
    - Set up the presolve environment and initialize necessary data structures.

- **repeat**
    - **Check rows:**
        - Remove all empty rows.
        - Remove redundant or dominated rows.
        - Handle singleton rows:
            - Eliminate rows where a single variable can be determined directly.
            - Process forcing rows that enforce specific variable values.

    - **Check columns:**
        - Remove all empty columns.
        - Remove fixed columns (where lower and upper bounds are equal).
        - Handle singleton columns:
            - Process and remove columns with only one non-zero coefficient.
            - Simplify doubleton equations (involving two variables).
            - Detect and remove implied free columns (where the variable is unconstrained).

    - **Bound tightening:**
        - Propagate constraints to tighten bounds on variables.
        - Apply dual inference to tighten bounds and check for infeasibility.

    - **Duplicate detection:**
        - Identify and remove duplicate rows.
        - Identify and remove duplicate columns.

    - **Matrix enhancement:**
        - Adjust the constraint matrix to improve sparsity and reduce complexity.

    - **Feasibility check:**
        - Detect infeasibility or unboundedness early in the process.

- **until** no further reductions can be made.

- **Postsolve:**
    - Apply postsolve procedures to recover the original problem's solution from the reduced problem.
    - Ensure that the final solution is both feasible and optimal for the original problem.

**end GLPK Presolve.**
<sup id="cite4">[Makhorin, 2021](#ref4)</sup>

</details>

<details>
    <summary><h3 id="highs-presolve-routine"> HiGHs Presolve Routine </h3></summary>

**Procedure HiGHS Presolve**
- **Initialization:**
    - Set up necessary data structures and configurations for presolve.

- **repeat**
    - **Check rows:**
        - Remove all empty rows.
        - Remove redundant rows (e.g., rows that are duplicates or dominated).
        - Handle singleton rows:
            - Eliminate rows where a single variable can be determined directly.
            - Process forcing rows (e.g., rows that force a variable to a certain value).

    - **Check columns:**
        - Remove all empty columns.
        - Remove fixed columns (where lower and upper bounds are equal).
        - Handle singleton columns:
            - Free column singletons: Remove or simplify columns with a single non-zero entry.
            - Column singletons in doubleton equations: Simplify or remove columns involved in two-variable equations.
            - Implied free column singletons: Detect and remove columns where the bounds imply that the variable can take on any value.

    - **Implied bounds:**
        - Tighten bounds on variables using implications from other constraints.
        - Apply bound tightening techniques to reduce the feasible region and simplify the problem.

    - **Matrix modifications:**
        - Apply rules to eliminate dominated rows and columns.
        - Modify the constraint matrix to enhance sparsity and reduce complexity.

    - **Detect infeasibilities and unboundedness:**
        - Check if the problem is infeasible or unbounded early in the process.
        - If detected, terminate the presolve and report the issue.

    - **Postsolve preparation:**
        - Store necessary data for reconstructing the original problem solution after solving the reduced problem.
        - Ensure that the solution to the reduced problem can be used to recover the solution to the original problem.

- **until** no reductions in the last pass.

- **Postsolve:**
    - Apply postsolve procedures to recover the original problem's solution from the reduced problem.
    - Ensure that the final solution satisfies feasibility and optimality conditions.
    - Perform additional simplex iterations if needed to refine the solution after postsolve.

**end HiGHS Presolve.**
<sup id="cite3">[Galabova, 2022](#ref3)</sup>

```mermaid
graph TD
    A[Start] --> B[Remove all fixed variables]
    B --> C{No reductions in last pass?}
    C --> |No| D[Check Rows]
    D --> E[Remove all empty rows]
    E --> F[Remove redundant rows]
    F --> G[Process singleton rows]
    G --> H[Check Columns]
    H --> I[Remove all empty columns]
    I --> J[Remove fixed columns]
    J --> K[Process singleton columns]
    K --> L[Simplify doubleton equations]
    L --> M[Bound Tightening]
    M --> N[Remove Dominated Constraints]
    N --> O[Check for Duplicates]
    O --> P[Remove duplicate rows]
    P --> Q[Remove duplicate columns]
    Q --> R[Handle Implied Free and Forcing Variables]
    R --> S{Infeasibility Detected?}
    S --> |Yes| T[End]
    S --> |No| C
    C --> |Yes| U[Remove all empty rows and columns]
    U --> V[End]
```


</details>



<p align="right">(<a href="#readme-top">back to top</a>)</p>

<details>
  <summary><h2>Presolve Methods Checklist</h2></summary>

- [ ] **Fixed Variable Detection**
  - Identify and remove variables with the same upper and lower bounds.

- [x] **Row Singleton Removal**
  - Identify and remove rows with a single non-zero element.

- [ ] **Column Singleton Removal**
  - Identify and remove columns with a single non-zero element.

- [ ] **Forcing Constraints Removal**
  - Remove constraints that force certain variable values.

- [ ] **Dominated Constraints Removal**
  - Identify and remove constraints dominated by other constraints.

- [x] **Free and Implied Free Column Singleton Removal**
  - Identify and remove free column singletons and implied free column singletons.

- [ ] **Doubleton Equation Handling**
  - Process and potentially remove column singletons in combination with a doubleton equation.

- [x] **Duplicate Row Removal**
  - Identify and remove duplicate rows.

- [ ] **Duplicate Column Removal**
  - Identify and remove duplicate columns.

- [ ] **Bound Tightening**
  - Tighten bounds using dual information, constraint propagation, and implications from other constraints.

- [ ] **Implied Equality Detection**
  - Detect and process implied equalities.

- [ ] **Sparsity Enhancement**
  - Enhance matrix sparsity through aggregation and other techniques.

- [ ] **Scaling**
  - Apply row and column scaling for numerical stability.

- [ ] **Infeasibility Detection**
  - Check if the problem is infeasible during the presolve process.

- [ ] **Matrix Enhancement**
  - Modify the constraint matrix to reduce complexity and improve numerical stability.

- [ ] **Postsolve Preparation**
  - Store necessary data for reconstructing the original problem solution after solving the reduced problem.

- [ ] **Postsolve**
  - Apply procedures to recover the original problem's solution from the reduced problem, ensuring feasibility and optimality.


</details>

## 

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Packages

```julia
import Pkg; Pkg.add("SuiteSparseQR")

In [12]:
using LinearAlgebra
using SparseArrays
using DataStructures
using Random
using ArgParse

# local modules
push!(LOAD_PATH, realpath("../src"))
# using LpConstants
# using LpUtils
using LpProblem
using LpReadMPS
using LpPresolve

using BenchmarkTools
using Test

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Modified LPProblem Structs
Below I am experimenting with different formulations of the LPProblem to capture sufficient information for post-processing a reduced LP problem.

<details>
    <summary><h3>PreprocessedLPProblem</h3></summary>

```julia
struct PreprocessedLPProblem
    original_problem::LPProblem  # The original LP problem before preprocessing
    reduced_problem::LPProblem   # The reduced LP problem after preprocessing
    removed_rows::Vector{Int}    # Indices of removed rows
    removed_cols::Vector{Int}    # Indices of removed columns (if applicable)
    row_ratios::Dict{Int, Tuple{Int, Float64}}  # Mapping of removed rows to their corresponding row and ratio
end
```

</details>


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Test Problems

Here are some test problems that are used to check various aspects of the presolver.

In [ ]:
mps_filename = "../check/problems/mps_files/test.mps"
lp = read_mps_from_file(mps_filename)
print(lp)

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Presolve Features


### Fixed varible detection

<details>
    <summary> Function </summary>

```julia
function lp_detect_and_remove_fixed_variables(lp_model::PreprocessedLPProblem; ε::Float64 = 1e-8, verbose::Bool = false)
    # Unpack the original and reduced problems
    original_lp = lp_model.original_problem
    reduced_lp = lp_model.reduced_problem
    removed_rows = lp_model.removed_rows
    removed_cols = lp_model.removed_cols
    row_ratios = lp_model.row_ratios
    var_solutions = lp_model.var_solutions
    row_scaling = lp_model.row_scaling
    col_scaling = lp_model.col_scaling
    is_infeasible = lp_model.is_infeasible

    # Detect fixed variables (where l == u)
    fixed_vars = [i for i in 1:length(reduced_lp.vars) if abs(reduced_lp.l[i] - reduced_lp.u[i]) < ε]
    remaining_vars_indices = setdiff(1:length(reduced_lp.vars), fixed_vars)

    # Store the fixed variable names and their solved values in var_solutions
    for i in fixed_vars
        var_solutions[reduced_lp.vars[i]] = reduced_lp.l[i]  # Store the variable name and its fixed value
    end

    # Debug statements
    if verbose
        println("#" ^ 80)
        println("~" ^ 80)
        println("Fixed Variable Detection")
        println("~" ^ 80)
        println("Total number of variables: ", length(reduced_lp.vars))
        println("Fixed variables: ", fixed_vars)
        println("Remaining variables: ", remaining_vars_indices)
        println("The objective coefficients after removal: ", reduced_lp.c[remaining_vars_indices])
        println("Fixed variable values (var_solutions): ", var_solutions)
        println("~" ^ 80)
        println("#" ^ 80)
        println()
    end

    if isempty(fixed_vars)
        # If no fixed variables are found, return the original problem unchanged
        return lp_model
    end

    # Adjust the right-hand side b by subtracting the contribution of the fixed variables
    contribution = reduced_lp.A[:, fixed_vars] * reduced_lp.l[fixed_vars]  # This returns a vector
    new_b = reduced_lp.b .- contribution  # Subtract the contribution from b

    # Create new LPProblem without fixed variables
    new_A = reduced_lp.A[:, remaining_vars_indices]
    new_c = reduced_lp.c[remaining_vars_indices]
    new_l = reduced_lp.l[remaining_vars_indices]
    new_u = reduced_lp.u[remaining_vars_indices]
    new_vars = reduced_lp.vars[remaining_vars_indices]
    new_constraint_types = reduced_lp.constraint_types
    new_variable_types = reduced_lp.variable_types[remaining_vars_indices]  # Ensure variable types are adjusted too

    # Check for infeasibility after adjusting constraints
    is_infeasible = false
    for i in 1:length(new_b)
        if new_constraint_types[i] == 'E' && norm(new_A[i, :], Inf) < ε
            if abs(new_b[i]) > ε
                is_infeasible = true
                if verbose
                    println("Infeasibility detected in constraint $i after removing fixed variables.")
                end
                break
            end
        end
    end

    # Remove zero rows if any
    zero_row_indices = [i for i in 1:size(new_A, 1) if norm(new_A[i, :], Inf) < ε]
    if !isempty(zero_row_indices)
        if verbose
            println("Removing zero rows: ", zero_row_indices)
        end
        new_A = new_A[setdiff(1:end, zero_row_indices), :]
        new_b = new_b[setdiff(1:end, zero_row_indices)]
        new_constraint_types = new_constraint_types[setdiff(1:end, zero_row_indices)]
        # Update removed_rows
        removed_rows = vcat(removed_rows, zero_row_indices)
    end

    # Create the reduced LP problem
    new_reduced_lp = LPProblem(
        reduced_lp.is_minimize,
        new_c,
        new_A,
        new_b,
        new_constraint_types,  # Updated order
        new_l,
        new_u,
        new_vars,
        new_variable_types  # Include variable types in the new LPProblem
    )

    # Return updated PreprocessedLPProblem
    return PreprocessedLPProblem(
        original_lp,
        new_reduced_lp,
        removed_rows,
        vcat(removed_cols, fixed_vars),
        row_ratios,
        var_solutions,
        row_scaling,
        col_scaling,
        is_infeasible
    )
end
```

</details>

In [ ]:
# Define an LP problem as per the example provided
c = [1.0, 2.0, 4.0]  # Objective function coefficients
A = sparse([1.0 -3.0 0.0; 2.0 1.0 -5.0])  # Constraint matrix
b = [10.0, 15.0]  # Right-hand side
l = [12.0, 0.0, 2.0]  # Lower bounds (x1 is fixed, x3 is fixed)
u = [12.0, 1.0, 2.0]  # Upper bounds
vars = ["x1", "x2", "x3"]  # Variable names
constraint_types = ['L', 'L']  # Constraint types (all less than or equal)
variable_types = [:Continuous, :Continuous, :Continuous]  # Variable types

# Create the original LPProblem
lp_original = LPProblem(
    false,  # Maximization problem
    c,      # Objective coefficients
    A,      # Constraint matrix
    b,      # Right-hand side
    constraint_types,  # Constraint types
    l,      # Lower bounds
    u,      # Upper bounds
    vars,   # Variable names
    variable_types  # Variable types
)

# Create the PreprocessedLPProblem with the same original and reduced problem (initially identical)
preprocessed_lp = PreprocessedLPProblem(
    lp_original,              # The original problem
    lp_original,              # The reduced problem (initially same as original)
    Int[],                    # No removed rows initially
    Int[],                    # No removed columns initially
    Dict{Int, Tuple{Int, Float64}}(),  # No row ratios
    Dict{String, Float64}(),  # Empty var_solutions dictionary
    Vector{Float64}(),        # Initialize row_scaling as an empty vector
    Vector{Float64}(),        # Initialize col_scaling as an empty vector
    false                     # Initially not infeasible
)

# Run the detect_and_remove_fixed_variables function
preprocessed_lp_after = lp_detect_and_remove_fixed_variables(preprocessed_lp; verbose = true)

# Output the reduced problem to check if fixed variables were correctly removed and their values stored
println("Original Problem: ", lp_original)
println("Reduced Problem after fixed variable removal: ", preprocessed_lp_after.reduced_problem)
println("Removed Columns (fixed variables): ", preprocessed_lp_after.removed_cols)
println("Variable solutions (var_solutions): ", preprocessed_lp_after.var_solutions)


In [ ]:
preprocessed_lp_after.reduced_problem.A

### Removing Empty Rows

This is the first and simplest presolving feature.


<details>
    <summary> First attempt</summary>

```julia
function lp_remove_zero_rows(lp::LPProblem; ε::Float64=1e-8)
    # Find non-zero rows
    non_zero_rows = [i for i in 1:size(lp.A, 1) if any(abs.(lp.A[i, :]) .> ε)]
    println(non_zero_rows)
    removed_rows = setdiff(1:size(lp.A, 1), non_zero_rows)
    println(removed_rows)
    
    # Initialize the dictionary to store the ratios of removed rows
    row_ratios = Dict{Int, Tuple{Int, Float64}}()

    # Assign a ratio of 0.0 for all removed rows to indicate no values
    for removed_row in removed_rows
        row_ratios[removed_row] = (removed_row, 0.0)
    end

    # Create new LPProblem with non-zero rows
    new_A = lp.A[non_zero_rows, :]
    new_b = lp.b[non_zero_rows]
    new_constraint_types = lp.constraint_types[non_zero_rows]

    # Construct the reduced LPProblem
    reduced_lp = LPProblem(lp.is_minimize, lp.c, new_A, new_b, lp.l, lp.u, lp.vars, new_constraint_types)

    # Return the PreprocessedLPProblem struct containing the original and reduced LPProblem, removed rows, and ratios
    return PreprocessedLPProblem(lp, reduced_lp, removed_rows, Int[], row_ratios)
end
```
<h3>Test Case</h3>

```julia
# Create the test problem
lp = create_test_problem_with_empty_rows()
# lp = create_larger_test_problem()

# Run the function to remove zero rows and preprocess the LP problem
preprocessed_lp = lp_remove_zero_rows(lp)

# Display the reduced problem and the removed rows with their ratios
println("Reduced A matrix:")
display(preprocessed_lp.reduced_problem.A)

println("Reduced b vector:")
display(preprocessed_lp.reduced_problem.b)

println("Removed rows and their ratios:")
for (removed_row, (base_row, ratio)) in preprocessed_lp.row_ratios
    println("Row $removed_row was removed. It is assigned a ratio of $ratio.")
end
```

</details>

<details>
    <summary> Function </summary>

```julia
function lp_remove_zero_rows(preprocessed_problem::PreprocessedLPProblem; ε::Float64 = 1e-8, verbose::Bool = false)
    # Unpack problem
    original_lp = preprocessed_problem.original_problem
    reduced_lp = preprocessed_problem.reduced_problem
    removed_rows = preprocessed_problem.removed_rows
    removed_cols = preprocessed_problem.removed_cols
    row_ratios = preprocessed_problem.row_ratios
    var_solutions = preprocessed_problem.var_solutions
    row_scaling = preprocessed_problem.row_scaling
    col_scaling = preprocessed_problem.col_scaling
    is_infeasible = preprocessed_problem.is_infeasible

    A = reduced_lp.A
    b = reduced_lp.b
    constraint_types = reduced_lp.constraint_types

    # Efficiently find non-zero rows using the norm of each row
    non_zero_rows = []
    zero_rows = []

    for i in 1:size(A, 1)
        if norm(A[i, :], Inf) < ε
            push!(zero_rows, i)
        else
            push!(non_zero_rows, i)
        end
    end

    # Debug statements
    if verbose 
        println("#" ^ 80)
        println("~" ^ 80)
        println("Remove Zero Rows Function")
        println("~" ^ 80)
        println("Total number of rows: ", size(reduced_lp.A, 1))
        println("Non-zero rows: ", non_zero_rows)
        println("Removed zero rows: ", zero_rows)
        println("~" ^ 80)
        println("#" ^ 80)
        println()
    end

    # Check for infeasibility in zero rows
    for idx in zero_rows
        if constraint_types[idx] == 'E' && abs(b[idx]) > ε
            is_infeasible = true
            if verbose
                println("Infeasibility detected due to zero row at index $idx with non-zero RHS.")
            end
            break
        end
    end

    if is_infeasible
        # Return early if problem is infeasible
        return PreprocessedLPProblem(
            original_lp,
            reduced_lp,
            vcat(removed_rows, zero_rows),
            removed_cols,
            row_ratios,
            var_solutions,
            row_scaling,
            col_scaling,
            true  # is_infeasible set to true
        )
    end

    # Adjust the problem based on non-zero rows
    new_A = A[non_zero_rows, :]
    new_b = b[non_zero_rows]
    new_constraint_types = constraint_types[non_zero_rows]

    # Construct the reduced LPProblem (variables remain unchanged)
    new_reduced_lp = LPProblem(
        reduced_lp.is_minimize, 
        reduced_lp.c, 
        new_A, 
        new_b, 
        new_constraint_types, 
        reduced_lp.l, 
        reduced_lp.u, 
        reduced_lp.vars, 
        reduced_lp.variable_types
    )

    # Return the updated PreprocessedLPProblem struct
    return PreprocessedLPProblem(
        original_lp,
        new_reduced_lp,
        vcat(removed_rows, zero_rows),
        removed_cols,  # Variables aren't removed in this process
        row_ratios,    # Update if necessary
        var_solutions,
        row_scaling,
        col_scaling,
        is_infeasible
    )
end
```

<\details>

In [ ]:
# Define a test LP problem
c_test = [1.0, -2.0, 3.0]  # Objective function coefficients
A_test = sparse([1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 0.0])  # Constraint matrix with a zero row
b_test = [5.0, 10.0, 0.0]  # Right-hand side (last row corresponds to a zero row)
l_test = [0.0, 0.0, 0.0]  # Lower bounds
u_test = [10.0, 10.0, 10.0]  # Upper bounds
vars_test = ["x1", "x2", "x3"]  # Variable names
constraint_types_test = ['L', 'L', 'L']  # All less than or equal constraints
variable_types_test = [:Continuous, :Continuous, :Continuous]  # Variable types

# Create the LPProblem instance for testing
test_problem = LPProblem(
    true,               # Minimize the objective
    c_test,             # Objective coefficients
    A_test,             # Constraint matrix
    b_test,             # Right-hand side
    constraint_types_test,  # Constraint types
    l_test,             # Lower bounds
    u_test,             # Upper bounds
    vars_test,          # Variable names
    variable_types_test # Variable types
)

# Preprocess the problem with some dummy removed rows and columns (for testing purposes)
preprocessed_problem = PreprocessedLPProblem(
    test_problem,       # Original problem
    test_problem,       # Reduced problem (initially same as original)
    Int[],              # No removed rows initially
    Int[],              # No removed columns initially
    Dict{Int, Tuple{Int, Float64}}(),  # No row ratios initially
    Dict{String, Float64}(),  # Empty variable solutions
    Vector{Float64}(),  # Empty row scaling
    Vector{Float64}(),  # Empty column scaling
    false               # No infeasibility detected initially
)

# Run the function to remove zero rows
updated_problem = lp_remove_zero_rows(preprocessed_problem, verbose=true)

# Display the updated problem
println("Updated problem:")
println("Removed rows: ", updated_problem.removed_rows)
println("Row ratios: ", updated_problem.row_ratios)
println("Reduced LP problem: ")
println("A (constraint matrix): ")
display(updated_problem.reduced_problem.A)
println("b (right-hand side): ", updated_problem.reduced_problem.b)
println("Constraint types: ", updated_problem.reduced_problem.constraint_types)
println(updated_problem)


## Removing Column Singletons

<details>
    <summary> Function </summary>

```julia
function lp_remove_row_singletons(lp_model::PreprocessedLPProblem; ε::Float64=1e-8, verbose::Bool = false)
    # Unpack problem
    original_lp = lp_model.original_problem
    reduced_lp = lp_model.reduced_problem
    removed_rows = lp_model.removed_rows
    removed_cols = lp_model.removed_cols
    row_ratios = lp_model.row_ratios
    var_solutions = lp_model.var_solutions
    is_infeasible = lp_model.is_infeasible

    # Create copies to modify
    A = copy(reduced_lp.A)
    b = copy(reduced_lp.b)
    c = copy(reduced_lp.c)
    l = copy(reduced_lp.l)
    u = copy(reduced_lp.u)
    vars = copy(reduced_lp.vars)
    variable_types = copy(reduced_lp.variable_types)
    constraint_types = copy(reduced_lp.constraint_types)

    # Initialize lists to keep track of removals
    rows_to_remove = Int[]
    removed_vars_indices = Int[]
    singleton_rows = Int[]

    # Identify and process row singletons
    for row in 1:size(A, 1)
        # Find non-zero coefficients in the row
        nz_indices = findnz(A[row, :])[1]
        if length(nz_indices) == 1
            push!(singleton_rows, row)
            var_index = nz_indices[1]
            coef = A[row, var_index]
            rhs = b[row]
            constr_type = constraint_types[row]

            if abs(coef) < ε
                continue  # Skip if coefficient is effectively zero
            end

            if constr_type == 'E'
                # Equality constraint: Fix the variable
                var_value = rhs / coef
                l[var_index] = var_value
                u[var_index] = var_value
                var_solutions[vars[var_index]] = var_value

                # Remove the variable and constraint from the problem
                push!(removed_vars_indices, var_index)
                push!(rows_to_remove, row)

            elseif constr_type == 'L'
                # Less-than-or-equal constraint: Adjust upper or lower bound
                new_bound = rhs / coef
                if coef > 0
                    u[var_index] = min(u[var_index], new_bound)
                else
                    l[var_index] = max(l[var_index], new_bound)
                end
                push!(rows_to_remove, row)

            elseif constr_type == 'G'
                # Greater-than-or-equal constraint: Adjust lower or upper bound
                new_bound = rhs / coef
                if coef > 0
                    l[var_index] = max(l[var_index], new_bound)
                else
                    u[var_index] = min(u[var_index], new_bound)
                end
                push!(rows_to_remove, row)
            else
                continue  # Skip other constraint types
            end

            # Check for infeasibility
            if l[var_index] > u[var_index] + ε
                is_infeasible = true
                if verbose
                    println("Infeasibility detected at variable $(vars[var_index]) after processing row singleton at row $row.")
                end
                break
            end
        end
    end

    if is_infeasible
        return PreprocessedLPProblem(
            original_lp,
            reduced_lp,
            vcat(removed_rows, rows_to_remove),
            vcat(removed_cols, removed_vars_indices),
            row_ratios,
            var_solutions,
            lp_model.row_scaling,
            lp_model.col_scaling,
            true
        )
    end

    # Remove rows and columns from the problem
    remaining_rows = setdiff(1:size(A, 1), rows_to_remove)
    remaining_cols = setdiff(1:size(A, 2), removed_vars_indices)

    # Compute remaining variables before modifying vars
    remaining_vars = vars[remaining_cols]

    # Verbose output matching previous functions
    if verbose
        println("#" ^ 80)
        println("~" ^ 80)
        println("Row Singleton Removal")
        println("~" ^ 80)
        println("Total number of constraints: ", size(reduced_lp.A, 1))
        println("Singleton rows identified: ", singleton_rows)
        println("Removed rows: ", rows_to_remove)
        println("Variables fixed (if any): ", [vars[i] for i in removed_vars_indices])
        println("Variable solutions (var_solutions): ", var_solutions)
        println("Remaining variables: ", remaining_vars)
        println("Remaining constraints: ", length(remaining_rows))
        println("~" ^ 80)
        println("#" ^ 80)
        println()
    end

    # Now modify A, b, c, l, u, vars, variable_types, constraint_types
    A = A[remaining_rows, remaining_cols]
    b = b[remaining_rows]
    constraint_types = constraint_types[remaining_rows]
    c = c[remaining_cols]
    l = l[remaining_cols]
    u = u[remaining_cols]
    vars = vars[remaining_cols]
    variable_types = variable_types[remaining_cols]

    # Construct the new LPProblem
    new_reduced_lp = LPProblem(
        reduced_lp.is_minimize,
        c,
        A,
        b,
        constraint_types,
        l,
        u,
        vars,
        variable_types
    )

    # Update removed rows and columns
    removed_rows = vcat(removed_rows, rows_to_remove)
    removed_cols = vcat(removed_cols, removed_vars_indices)

    # Return the updated PreprocessedLPProblem
    return PreprocessedLPProblem(
        original_lp,
        new_reduced_lp,
        removed_rows,
        removed_cols,
        row_ratios,
        var_solutions,
        lp_model.row_scaling,
        lp_model.col_scaling,
        is_infeasible
    )
end
```

</details>

In [ ]:
# Define a test LP problem with row singletons
c_test = [2.0, -3.0, 4.0, 1.0]  # Objective function coefficients
A_test = sparse([1.0 0.0 0.0 0.0;   # Row singleton
                 0.0 1.0 -1.0 0.0;   # Not a singleton
                 0.0 0.0 1.0 0.0;   # Row singleton
                 0.0 0.0 0.0 1.0])  # Row singleton
b_test = [10.0, 15.0, 20.0, 5.0]    # Right-hand side
l_test = [0.0, 0.0, 0.0, 0.0]       # Lower bounds
u_test = [100.0, 100.0, 100.0, 100.0]  # Upper bounds
vars_test = ["x1", "x2", "x3", "x4"]  # Variable names
constraint_types_test = ['E', 'L', 'E', 'E']  # Constraint types
variable_types_test = [:Continuous, :Continuous, :Continuous, :Continuous]  # Variable types

# Create the LPProblem instance for testing
test_problem = LPProblem(
    true,               # Minimize the objective
    c_test,             # Objective coefficients
    A_test,             # Constraint matrix
    b_test,             # Right-hand side
    constraint_types_test,  # Constraint types
    l_test,             # Lower bounds
    u_test,             # Upper bounds
    vars_test,          # Variable names
    variable_types_test # Variable types
)

# Preprocess the problem (initially identical)
preprocessed_problem = PreprocessedLPProblem(
    test_problem,       # Original problem
    test_problem,       # Reduced problem (initially same as original)
    Int[],              # No removed rows initially
    Int[],              # No removed columns initially
    Dict{Int, Tuple{Int, Float64}}(),  # No row ratios initially
    Dict{String, Float64}(),  # Empty variable solutions
    Vector{Float64}(),  # Empty row scaling
    Vector{Float64}(),  # Empty column scaling
    false               # No infeasibility detected initially
)

# Run the function to remove row singletons
updated_problem = lp_remove_row_singletons(preprocessed_problem; verbose = true)

# Display the updated problem
println("Updated problem:")
println("Removed rows: ", updated_problem.removed_rows)
println("Removed columns: ", updated_problem.removed_cols)
println("Variable solutions: ", updated_problem.var_solutions)
println("Reduced LP problem: ")
println("A: ")
display(updated_problem.reduced_problem.A)
println("b: ", updated_problem.reduced_problem.b)
println("c: ", updated_problem.reduced_problem.c)
println("l: ", updated_problem.reduced_problem.l)
println("u: ", updated_problem.reduced_problem.u)
println("vars: ", updated_problem.reduced_problem.vars)
println("Constraint types: ", updated_problem.reduced_problem.constraint_types)


### Removing Empty Columns

<details>
    <summary> Function </summary>

```julia
function lp_remove_zero_columns(preprocessed_lp::PreprocessedLPProblem; ε::Float64=1e-8, verbose::Bool=false)
    # Unpack the reduced problem
    reduced_lp = preprocessed_lp.reduced_problem
    A = reduced_lp.A
    c = reduced_lp.c
    l = reduced_lp.l
    u = reduced_lp.u
    vars = reduced_lp.vars
    variable_types = reduced_lp.variable_types
    var_solutions = copy(preprocessed_lp.var_solutions)
    is_minimize = reduced_lp.is_minimize
    is_infeasible = preprocessed_lp.is_infeasible  # Initialize is_infeasible

    # Efficiently find non-zero columns using sparse matrix properties
    col_nnz = diff(A.colptr)
    non_zero_columns = findall(col_nnz .> 0)
    zero_columns = findall(col_nnz .== 0)

    # Initialize lists for variables to remove and their solutions
    vars_to_remove = Int[]
    new_var_solutions = Dict{String, Float64}()

    # Process zero columns
    for idx in zero_columns
        var_name = vars[idx]
        lb = l[idx]
        ub = u[idx]
        coef = c[idx]

        # Assume we will remove this variable
        push!(vars_to_remove, idx)

        # Handle unbounded variables
        if isinf(lb) || isinf(ub)
            if is_minimize
                if (coef > 0 && isinf(lb)) || (coef < 0 && isinf(ub))
                    # Objective can decrease unboundedly
                    is_infeasible = true
                    break
                else
                    # Set variable to bound that optimizes objective
                    var_value = (coef ≥ 0) ? lb : ub
                end
            else
                # Maximization problem
                if (coef > 0 && isinf(ub)) || (coef < 0 && isinf(lb))
                    # Objective can increase unboundedly
                    is_infeasible = true
                    break
                else
                    # Set variable to bound that optimizes objective
                    var_value = (coef ≥ 0) ? ub : lb
                end
            end
        else
            # Variable is bounded
            # Set variable to bound that optimizes objective
            if is_minimize
                var_value = (coef ≥ 0) ? lb : ub
            else
                var_value = (coef ≥ 0) ? ub : lb
            end
        end

        # Record variable value if not infeasible
        if !is_infeasible
            new_var_solutions[var_name] = var_value
        end
    end

    if verbose
        println("#" ^ 80)
        println("~" ^ 80)
        println("Zero Column Removal")
        println("~" ^ 80)
        println("Total number of variables: ", length(vars))
        println("Zero columns identified: ", [vars[i] for i in zero_columns])
        println("Variables removed: ", [vars[i] for i in vars_to_remove])
        if is_infeasible
            println("Infeasibility detected during zero column removal.")
        else
            println("Variables fixed (if any): ", keys(new_var_solutions))
            println("Variable solutions (var_solutions): ", new_var_solutions)
            println("Remaining variables: ", vars[setdiff(1:end, vars_to_remove)])
        end
        println("~" ^ 80)
        println("#" ^ 80)
        println()
    end

    if is_infeasible
        # Return without modifying the problem further
        return PreprocessedLPProblem(
            preprocessed_lp.original_problem,
            reduced_lp,
            preprocessed_lp.removed_rows,
            vcat(preprocessed_lp.removed_cols, vars_to_remove),
            preprocessed_lp.row_ratios,
            var_solutions,  # Do not merge new_var_solutions as they may be invalid
            preprocessed_lp.row_scaling,
            preprocessed_lp.col_scaling,
            is_infeasible
        )
    end

    # Update variables by removing zero columns
    remaining_vars_indices = setdiff(1:length(vars), vars_to_remove)
    new_c = c[remaining_vars_indices]
    new_A = A[:, remaining_vars_indices]
    new_l = l[remaining_vars_indices]
    new_u = u[remaining_vars_indices]
    new_vars = vars[remaining_vars_indices]
    new_variable_types = variable_types[remaining_vars_indices]

    # Construct the new LPProblem
    new_reduced_lp = LPProblem(
        is_minimize,
        new_c,
        new_A,
        reduced_lp.b,
        reduced_lp.constraint_types,
        new_l,
        new_u,
        new_vars,
        new_variable_types
    )

    # Return the updated PreprocessedLPProblem struct
    return PreprocessedLPProblem(
        preprocessed_lp.original_problem,
        new_reduced_lp,
        preprocessed_lp.removed_rows,
        vcat(preprocessed_lp.removed_cols, vars_to_remove),
        preprocessed_lp.row_ratios,
        merge(var_solutions, new_var_solutions),
        preprocessed_lp.row_scaling,
        preprocessed_lp.col_scaling,
        is_infeasible
    )
end
```

<\details>

In [ ]:
# Define a test LP problem
c_test = [1.0, -2.0, 3.0, 0.0]  # Objective function coefficients (last variable is zero)
A_test = sparse([1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0])  # Constraint matrix (last column is zero)
b_test = [5.0, 10.0, 15.0]  # Right-hand side
l_test = [0.0, 0.0, 0.0, 0.0]  # Lower bounds
u_test = [10.0, 10.0, 10.0, 10.0]  # Upper bounds
vars_test = ["x1", "x2", "x3", "x4"]  # Variable names (x4 is a zero column)
constraint_types_test = ['L', 'L', 'L']  # All less than or equal constraints
variable_types_test = [:Continuous, :Continuous, :Continuous, :Continuous]  # Variable types

# Create the LPProblem instance for testing
test_problem = LPProblem(
    true,               # Minimize the objective
    c_test,             # Objective coefficients
    A_test,             # Constraint matrix
    b_test,             # Right-hand side
    constraint_types_test,  # Constraint types
    l_test,             # Lower bounds
    u_test,             # Upper bounds
    vars_test,          # Variable names
    variable_types_test # Variable types
)

# Preprocess the problem (initially identical)
preprocessed_problem = PreprocessedLPProblem(
    test_problem,       # Original problem
    test_problem,       # Reduced problem (initially same as original)
    Int[],              # No removed rows initially
    Int[],              # No removed columns initially
    Dict{Int, Tuple{Int, Float64}}(),  # No row ratios initially
    Dict{String, Float64}(),  # Empty variable solutions
    Vector{Float64}(),  # Empty row scaling
    Vector{Float64}(),  # Empty column scaling
    false               # No infeasibility detected initially
)

# Run the function to remove zero columns
updated_problem = lp_remove_zero_columns(preprocessed_problem; verbose = true)

# Display the updated problem
println("Updated problem:")
println("Removed columns: ", updated_problem.removed_cols)
println("Reduced LP problem: ")
println("A: ")
display(updated_problem.reduced_problem.A)
println("c: ", updated_problem.reduced_problem.c)
println("l: ", updated_problem.reduced_problem.l)
println("u: ", updated_problem.reduced_problem.u)
println("vars: ", updated_problem.reduced_problem.vars)
println("Constraint types: ", updated_problem.reduced_problem.constraint_types)


### Remove Linear Dependent Rows


<details>
    <summary> This first attempt </summary>

```julia
function lp_remove_linearly_dependent_rows(lp::LPProblem; ε::Float64=1e-8)
        # Create the augmented matrix [A b]
        augmented_matrix = hcat(lp.A, lp.b)
    
        rows_to_check = collect(1:size(augmented_matrix, 1))  # Start with all rows to check
        removed_rows = Vector{Int}()  # List of removed rows
        row_ratios = Dict{Int, Tuple{Int, Float64}}()  # Store ratios of removed rows
        
        while length(rows_to_check) > 1
            current_row_index = rows_to_check[1]
            current_row = augmented_matrix[current_row_index, :]
            
            for i in rows_to_check[2:end]
                compare_row = augmented_matrix[i, :]
                
                # Find the first non-zero index in the current_row
                non_zero_idx = findfirst(abs.(current_row) .> ε)
                
                if non_zero_idx !== nothing && compare_row[non_zero_idx] != 0
                    ratio = current_row[non_zero_idx] / compare_row[non_zero_idx]
                    
                    # Check if multiplying the compare_row by ratio gives the current_row
                    if all(abs.(current_row .- ratio .* compare_row) .< ε)
                        push!(removed_rows, i)  # Mark the row for removal
                        row_ratios[i] = (current_row_index, ratio)  # Store the row index and ratio
                    end
                end
            end
            
            rows_to_check = setdiff(rows_to_check, [current_row_index; removed_rows])  # Remove current and dependent rows
        end
        
        # Create the reduced matrix and vectors by excluding the removed rows
        non_removed_rows = setdiff(1:size(lp.A, 1), removed_rows)
        reduced_A = lp.A[non_removed_rows, :]
        reduced_b = lp.b[non_removed_rows]
        reduced_constraint_types = lp.constraint_types[non_removed_rows]
        
        # Construct the reduced LPProblem
        reduced_lp = LPProblem(
            lp.is_minimize,
            lp.c,
            reduced_A,
            reduced_b,
            lp.l,
            lp.u,
            lp.vars,
            reduced_constraint_types
        )
        
        # Return the PreprocessedLPProblem struct
        return PreprocessedLPProblem(lp, reduced_lp, removed_rows, Int[], row_ratios)
    end
end
```

<h2> Test problem </h2>

```julia
# Create the test problem
lp_test = create_test_problem_linear_dependence_rows()

# Run the function to preprocess the LP problem
preprocessed_lp = lp_remove_linearly_dependent_rows(lp_test)

# Display the reduced LP problem's A matrix and b vector
println("Reduced A matrix:")
display(preprocessed_lp.reduced_problem.A)

println("Reduced b vector:")
display(preprocessed_lp.reduced_problem.b)

println("Removed rows and their ratios:")
for (removed_row, (base_row, ratio)) in preprocessed_lp.row_ratios
    println("Row $removed_row was removed. It is a multiple of Row $base_row with a ratio of $ratio.")
end
```

</details>

<details>
     <summary> Function </summary>

```julia
function lp_remove_linearly_dependent_rows_test(preprocessed_lp::PreprocessedLPProblem; ε::Float64 = 1e-8, verbose::Bool = false)
    # Unpack the reduced problem
    reduced_lp = preprocessed_lp.reduced_problem

    # Create the augmented matrix [A b]
    augmented_matrix = hcat(reduced_lp.A, reduced_lp.b)

    rows_to_check = collect(1:size(augmented_matrix, 1))  # Start with all rows to check
    removed_rows = Vector{Int}()  # List of removed rows
    row_ratios = Dict{Int, Tuple{Int, Float64}}()  # Store ratios of removed rows

    while length(rows_to_check) > 1
        current_row_index = rows_to_check[1]
        current_row = augmented_matrix[current_row_index, :]

        for i in rows_to_check[2:end]
            compare_row = augmented_matrix[i, :]

            # Find the first non-zero index in the current_row
            non_zero_idx = findfirst(abs.(current_row) .> ε)

            if non_zero_idx !== nothing && compare_row[non_zero_idx] != 0
                ratio = current_row[non_zero_idx] / compare_row[non_zero_idx]

                # Check if multiplying the compare_row by ratio gives the current_row
                if all(abs.(current_row .- ratio .* compare_row) .< ε)
                    push!(removed_rows, i)  # Mark the row for removal
                    row_ratios[i] = (current_row_index, ratio)  # Store the row index and ratio
                end
            end
        end

        rows_to_check = setdiff(rows_to_check, [current_row_index; removed_rows])  # Remove current and dependent rows
    end

    # Sort row information
    removed_rows = sort(removed_rows)

    # Debug statements
    if verbose
        println("#" ^ 80)
        println("~" ^ 80)
        println("Remove Linearly Dependent Rows Function")
        println("~" ^ 80)
        println("Removed rows: ", removed_rows)
        println("Row Ratios: ", row_ratios)
        println("~" ^ 80)
        println("#" ^ 80)
        println()
    end

    # Create the reduced matrix and vectors by excluding the removed rows
    non_removed_rows = setdiff(1:size(reduced_lp.A, 1), removed_rows)
    reduced_A = reduced_lp.A[non_removed_rows, :]
    reduced_b = reduced_lp.b[non_removed_rows]
    reduced_constraint_types = reduced_lp.constraint_types[non_removed_rows]

    # Construct the reduced LPProblem
    reduced_lp_problem = LPProblem(
        reduced_lp.is_minimize,
        reduced_lp.c,
        reduced_A,
        reduced_b,
        reduced_constraint_types,
        reduced_lp.l,
        reduced_lp.u,
        reduced_lp.vars,
        reduced_lp.variable_types
    )

    # Return the updated PreprocessedLPProblem struct
    return PreprocessedLPProblem(
        preprocessed_lp.original_problem,
        reduced_lp_problem,
        vcat(preprocessed_lp.removed_rows, removed_rows),
        preprocessed_lp.removed_cols,
        merge(preprocessed_lp.row_ratios, row_ratios),
        preprocessed_lp.var_solutions,
        preprocessed_lp.row_scaling,
        preprocessed_lp.col_scaling,
        preprocessed_lp.is_infeasible
    )
end
```

</details>

In [ ]:
# Define a test LP problem with linearly dependent rows
c_test = [1.0, 2.0, 3.0]  # Objective function coefficients
A_test = sparse([1.0 2.0 3.0;   # First row
                 2.0 4.0 6.0;   # Linearly dependent on the first row
                 3.0 6.0 9.0;   # Linearly dependent on the first row
                 1.0 0.0 0.0])  # Independent row
b_test = [10.0, 20.0, 30.0, 5.0]  # Right-hand side
l_test = [0.0, 0.0, 0.0]       # Lower bounds
u_test = [100.0, 100.0, 100.0]  # Upper bounds
vars_test = ["x1", "x2", "x3"]  # Variable names
constraint_types_test = ['E', 'E', 'E', 'L']  # Equality and less-than constraints
variable_types_test = [:Continuous, :Continuous, :Continuous]  # Variable types

# Create the LPProblem instance for testing
test_problem = LPProblem(
    true,               # Minimize the objective
    c_test,             # Objective coefficients
    A_test,             # Constraint matrix
    b_test,             # Right-hand side
    constraint_types_test,  # Constraint types
    l_test,             # Lower bounds
    u_test,             # Upper bounds
    vars_test,          # Variable names
    variable_types_test # Variable types
) 

# Preprocess the problem (initially identical)
preprocessed_problem = PreprocessedLPProblem(
    test_problem,       # Original problem
    test_problem,       # Reduced problem (initially same as original)
    Int[],              # No removed rows initially
    Int[],              # No removed columns initially
    Dict{Int, Tuple{Int, Float64}}(),  # No row ratios initially
    Dict{String, Float64}(),  # Empty variable solutions
    Vector{Float64}(),  # Empty row scaling
    Vector{Float64}(),  # Empty column scaling
    false               # No infeasibility detected initially
)

# Run the function to remove linearly dependent rows
updated_problem = lp_remove_linearly_dependent_rows(preprocessed_problem; verbose = true)

# Display the updated problem
println("Updated problem:")
println("Removed rows: ", updated_problem.removed_rows)
println("Row Ratios: ", updated_problem.row_ratios)
println("Reduced LP problem: ")
println("A: ")
display(updated_problem.reduced_problem.A)
println("b: ", updated_problem.reduced_problem.b)
println("Constraint types: ", updated_problem.reduced_problem.constraint_types)


### Tightening Constraints 

In [20]:
# function remove_forcing_constraints(lp::LPProblem; ε::Float64=1e-8)
#     A, b, l, u = lp.A, lp.b, lp.l, lp.u
#     removed_rows = Int[]
#     row_ratios = Dict{Int, Tuple{Int, Float64}}()

#     # Iterate over each constraint
#     for i in 1:size(A, 1)
#         row = A[i, :]
#         lb, ub = 0.0, 0.0  # Bounds based on the type of constraint

#         if lp.constraint_types[i] == '<' || lp.constraint_types[i] == '≤'
#             ub = b[i]
#         elseif lp.constraint_types[i] == '>' || lp.constraint_types[i] == '≥'
#             lb = b[i]
#         elseif lp.constraint_types[i] == '='
#             lb = b[i]
#             ub = b[i]
#         end

#         # Calculate minimum and maximum possible values for the row
#         # min_row_value = sum(row[j] * (row[j] > 0 ? l[j] : u[j]) for j in 1:size(A, 2))
#         # max_row_value = sum(row[j] * (row[j] > 0 ? u[j] : l[j]) for j in 1:size(A, 2))
#         # min_row_value = sum(A[i, j] * l[j] for j in 1:size(A, 2))       
#         # max_row_value = sum(A[i, j] * u[j] for j in 1:size(A, 2))
#         # min_row_value = dot(A[i, :], l)
#         # max_row_value = dot(A[i, :], u)
#         # min_row_value = clamp(dot(A[i, :], l), 0, Inf)
#         # max_row_value = clamp(dot(A[i, :], u), 0, Inf)
#         min_row_value = minimum(A[i, :] .* l)
#         max_row_value = maximum(A[i, :] .* u)
        

#         println("Constraint $i:")
#         println("A: ", A[i, :])
#         println("l: ", l)
#         println("u: ", u)
#         println("  min_row_value: $min_row_value")
#         println("  max_row_value: $max_row_value")
#         # println("  ub: $ub")
#         # println("  lb: $lb")
#         # println("  ε: $ε")

#         if min_row_value > ub + ε || max_row_value < lb - ε
#             push!(removed_rows, i)
#             row_ratios[i] = (i, 0.0)
#         end
#     end

#     # Create the reduced A matrix and b vector by removing forcing constraints
#     non_removed_rows = setdiff(1:size(A, 1), removed_rows)
#     reduced_A = A[non_removed_rows, :]
#     reduced_b = b[non_removed_rows]
#     reduced_constraint_types = lp.constraint_types[non_removed_rows]

#     println(non_removed_rows)
#     println(reduced_A)
#     println(reduced_b)
#     println(reduced_constraint_types)

#     # Construct the reduced LPProblem
#     reduced_lp = LPProblem(lp.is_minimize, lp.c, reduced_A, reduced_b, lp.l, lp.u, lp.vars, reduced_constraint_types)

#     return PreprocessedLPProblem(lp, reduced_lp, removed_rows, Int[], row_ratios)
# end


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## General Presolve Routine

<details>
    <summary> Currently this is just copy and pasted from Harleys lp_llama_ipm.ipynb notebook.</summary>

```julia
function presolve(lp::LPProblem; eps::Float64=1e-8)
    is_minimize, c, A, b = lp.is_minimize, lp.c, lp.A, lp.b
    l, u = copy(lp.l), copy(lp.u)  # Create copies to avoid modifying the original bounds
    vars, constraint_types = lp.vars, lp.constraint_types
    m, n = size(A)

    # Initialize masks for rows and columns to keep
    keep_rows = trues(m)
    keep_cols = trues(n)

    # Step 1: Remove zero columns
    for j in 1:n
        if nnz(A[:, j]) == 0 && abs(c[j]) < eps
            keep_cols[j] = false
        end
    end

    # Step 2: Remove zero rows
    for i in 1:m
        if nnz(A[i, :]) == 0
            if abs(b[i]) < eps
                keep_rows[i] = false
            else
                error("Infeasible problem: zero row with non-zero RHS")
            end
        end
    end

    # Step 3: Remove duplicate rows
    for i in 1:m
        if !keep_rows[i]
            continue
        end
        for j in (i+1):m
            if !keep_rows[j]
                continue
            end
            if A[i, :] == A[j, :] && abs(b[i] - b[j]) < eps && constraint_types[i] == constraint_types[j]
                keep_rows[j] = false
            elseif A[i, :] == -A[j, :] && abs(b[i] + b[j]) < eps &&
                    ((constraint_types[i] == 'L' && constraint_types[j] == 'G') ||
                     (constraint_types[i] == 'G' && constraint_types[j] == 'L'))
                keep_rows[j] = false
            end
        end
    end

    # Step 4 - Conservative bound tightening with feasibility checks - k is the selected column
    bound_updates = true
    iteration = 1
    while bound_updates
        bound_updates = false
        println("\nBound tightening loop: iteration $iteration")

        lower_bounds = [Float64[] for _ in 1:n]
        upper_bounds = [Float64[] for _ in 1:n]  

        for i in 1:m  # Loop over constraints
            if !keep_rows[i]
                continue
            end

            println("\nAnalyzing bounds for constraint $i:")
            row = A[i, :]  # Get the i-th row            

            for k in 1:n  # select variable column
                if (A[i,k] == 0) || !keep_cols[k]
                    continue
                end

                lower_sum = sum((l[j] * aij for (j, aij) in zip(row.nzind, row.nzval) if j != k && aij > 0), init=0.0)
                upper_sum = sum((u[j] * aij for (j, aij) in zip(row.nzind, row.nzval) if j != k && aij < 0), init=0.0)
                new_bound = (b[i] - lower_sum - upper_sum)/ A[i,k]
                println("  analyzing bounds for variable $(vars[k]): k = $(k): A[i,k] = $(A[i,k]), b[i] = $(b[i]), lower_sum = $lower_sum, upper_sum = $upper_sum, new_bound = $new_bound")
                
                if constraint_types[i] == 'L'
                    if A[i,k] > 0
                        push!(upper_bounds[k], new_bound)
                    else
                        push!(lower_bounds[k], new_bound)
                    end
                elseif constraint_types[i] == 'G'
                    if A[i,k] > 0
                        push!(lower_bounds[k], new_bound)
                    else
                        push!(upper_bounds[k], new_bound)
                    end
                elseif constraint_types[i] == 'E'
                    push!(lower_bounds[k], new_bound)
                    push!(upper_bounds[k], new_bound)
                end

                if !isempty(lower_bounds[k])
                    if length(lower_bounds[k]) > 0
                        new_lower = maximum(lower_bounds[k])
                        if new_lower > l[k] + eps
                            println("variable $(vars[k]) tightening lower bound: $new_lower (prev: $(l[k]))")
                            l[k] = new_lower
                            bound_updates = true                    
                        end
                    end
                end
                
                if !isempty(upper_bounds)
                    if length(upper_bounds[k]) > 0
                        new_upper = minimum(upper_bounds[k])
                        if new_upper < u[k] - eps
                            println("variable $(vars[k]) tightening upper bound: $new_upper (prev: $(u[k]))")
                            u[k] = new_upper
                            bound_updates = true
                        end
                    end
                end
                
                # Ensure lower bound doesn't exceed upper bound
                if l[k] > u[k] + eps
                    println("WARNING: variable $(vars[k]) lower bound ($(l[k])) exceeds upper bound ($(u[k])) for $(vars[k]) - adjusting bounds to maintain feasibility")
                    avg_bound = (l[k] + u[k]) / 2
                    l[k] = avg_bound - eps
                    u[k] = avg_bound + eps
                end
                println("calculated bounds for $(vars[k]): [$(l[k]), $(u[k])]")                
            end
        end
           
        if bound_updates
            iteration += 1
        else
            println("No further bound updates possible")
        end
    end

    # Step 5: Fix variables and tighten bounds
    fixed_vars = Dict{String, Float64}()
    for j in 1:n
        col = A[:, j]
        if nnz(col) == 1
            i = findfirst(!iszero, col)
            if abs(col[i]) == 1 && constraint_types[i] == 'E'
                val = b[i] / col[i]
                if l[j] <= val && val <= u[j]
                    fixed_vars[vars[j]] = val
                    keep_cols[j] = false
                    b .-= val * col
                end
            end
        end
    end

    # Apply the reductions
    A_new = A[keep_rows, keep_cols]
    b_new = b[keep_rows]
    c_new = c[keep_cols]
    l_new = l[keep_cols]
    u_new = u[keep_cols]
    vars_new = vars[keep_cols]
    constraint_types_new = constraint_types[keep_rows]

    # Adjust the objective for fixed variables
    obj_adjust = 0.0
    for (j, var) in enumerate(vars)
        if haskey(fixed_vars, var)
            obj_adjust += c[j] * fixed_vars[var]
        end
    end

    if options[OPTION_VERBOSE]
        println("\nPresolve summary:")
        println("  Original problem size: $(m) x $(n)")
        println("  Reduced problem size: $(sum(keep_rows)) x $(sum(keep_cols))")
        println("  Number of fixed variables: $(length(fixed_vars))")
        println("  Objective adjustment: $obj_adjust")
        println("  Updated bounds:")
        for (var, lb, ub) in zip(vars_new, l_new, u_new)
            println("    $var: [$lb, $ub]")
        end
    end

    return LPProblem(is_minimize, c_new, Matrix(A_new), b_new, l_new, u_new, vars_new, constraint_types_new), fixed_vars, obj_adjust
end
```

</details>

<details> 
    <summary> Function </summary>

```julia
function presolve_lp(lp_problem::LPProblem; verbose::Bool = false)
    # Initialize PreprocessedLPProblem
    preprocessed_lp = PreprocessedLPProblem(
        lp_problem,       # Original problem
        lp_problem,       # Reduced problem (initially same as original)
        Int[],            # No removed rows initially
        Int[],            # No removed columns initially
        Dict{Int, Tuple{Int, Float64}}(),  # No row ratios initially
        Dict{String, Float64}(),  # Empty variable solutions
        Vector{Float64}(),  # Empty row scaling
        Vector{Float64}(),  # Empty column scaling
        false               # No infeasibility detected initially
    )

    # Apply preprocessing methods
    preprocessed_lp = lp_detect_and_remove_fixed_variables(preprocessed_lp; verbose=verbose)
    preprocessed_lp = lp_remove_zero_rows(preprocessed_lp; verbose=verbose)
    preprocessed_lp = lp_remove_row_singletons(preprocessed_lp; verbose=verbose)
    preprocessed_lp = lp_remove_zero_columns(preprocessed_lp; verbose=verbose)
    preprocessed_lp = lp_remove_linearly_dependent_rows(preprocessed_lp; verbose=verbose)

    return preprocessed_lp
end
```

</details>

In [ ]:
# Define a test LP problem
c_test = [1.0, 0.0, 3.0, 0.0]  # Objective function coefficients (some columns are zero)
A_test = sparse([1.0 0.0 2.0 0.0;    # Row 1: has a zero column
                 0.0 0.0 0.0 0.0;    # Row 2: zero row (to be removed)
                 2.0 0.0 4.0 0.0;    # Row 3: linearly dependent on row 1
                 0.0 0.0 0.0 1.0])   # Row 4: independent variable
b_test = [10.0, 0.0, 20.0, 5.0]      # Right-hand side (last column has a zero column)
l_test = [0.0, 0.0, 0.0, 0.0]        # Lower bounds
u_test = [100.0, 100.0, 100.0, 100.0]  # Upper bounds
vars_test = ["x1", "x2", "x3", "x4"]  # Variable names
constraint_types_test = ['L', 'E', 'L', 'E']  # Constraints: mix of equality and less-than
variable_types_test = [:Continuous, :Continuous, :Continuous, :Continuous]  # Variable types

# Create the LPProblem instance for testing
test_problem = LPProblem(
    true,               # Minimize the objective
    c_test,             # Objective coefficients
    A_test,             # Constraint matrix
    b_test,             # Right-hand side
    constraint_types_test,  # Constraint types
    l_test,             # Lower bounds
    u_test,             # Upper bounds
    vars_test,          # Variable names
    variable_types_test # Variable types
)

# Call the presolve function
preprocessed_lp = presolve_lp(test_problem; verbose=true)

# Display the updated problem
println("Preprocessed LP Problem:")
println("Removed rows: ", preprocessed_lp.removed_rows)
println("Removed columns: ", preprocessed_lp.removed_cols)
println("Row Ratios: ", preprocessed_lp.row_ratios)
println("Variable solutions: ", preprocessed_lp.var_solutions)
println("Reduced LP problem: ")
println("A: ")
display(preprocessed_lp.reduced_problem.A)
println("b: ", preprocessed_lp.reduced_problem.b)
println("c: ", preprocessed_lp.reduced_problem.c)
println("l: ", preprocessed_lp.reduced_problem.l)
println("u: ", preprocessed_lp.reduced_problem.u)
println("vars: ", preprocessed_lp.reduced_problem.vars)
println("Constraint types: ", preprocessed_lp.reduced_problem.constraint_types)



<p align="right">(<a href="#readme-top">back to top</a>)</p>

## References

1. <a id="ref1"></a> Andersen, E.D., Andersen, K.D. Presolving in linear programming. *Mathematical Programming* 71, 221–245 (1995). [https://doi.org/10.1007/BF01586000](https://doi.org/10.1007/BF01586000)

2. <a id="ref2"></a> Eikland, K., & Notebaert, P. (2020). *lp_solve: Open source (Mixed-Integer) Linear Programming system* (Version 5.5). Retrieved from http://lpsolve.sourceforge.net/5.5/

3. <a id="ref3"></a> Galabova, I. *Presolve, crash and software engineering for HiGHS* (Master's thesis, University of Edinburgh, 2022). Retrieved from https://era.ed.ac.uk/handle/1842/39725

4. <a id="ref4"></a> Makhorin, A. *GLPK (GNU Linear Programming Kit), version 5.0*. (2021). Retrieved from https://www.gnu.org/software/glpk/

